In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
#leer la bd
df = pd.read_csv("../../data/processed/BD_ID_CLIENTE_N.POLIZA_X_MES.csv", index_col="Unnamed: 0")
df.head()

,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01
1690,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9860,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [3]:
#agrego la bd de datos cliente
df_cli = pd.read_csv("../../data/processed/BD_ID_CLIENTE_D.Pers_Modo.Pago.csv", index_col="Unnamed: 0")
df_cli.set_index("CUSTOMER_ID", inplace=True)
df_cli.sort_index(inplace=True)
df_cli.head()

,DATE_OF_BIRTH,GENDER,MARITAL_STATUS,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad,ModoPago,edad_r
CUSTOMER_ID,,,,,,,,,,,,
1690,1943-03-06,M,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
5575,1966-06-10,M,Casado,NaN,NaN,ESCB,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2017,51.0,Mix,4.0
7968,1911-01-11,M,Casado,NaN,NaN,NaN,NaN,NaN,31/01/2018,107.0,CC,0.0
9028,1961-07-15,M,Casado,NaN,NaN,ARQU,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2018,56.0,Mix,4.0
9860,1967-04-06,M,Soltero,NaN,NaN,EMPL,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2016,50.0,Mix,4.0


# armando bloques 9 m (6 train, 3 target)

### Creo lista de dataframes

In [4]:
df = df.iloc[:,:-2]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 1690 to 21260067
Columns: 165 entries, 2004-03 to 2017-11
dtypes: float64(165)
memory usage: 63.3 MB


In [5]:
len(df.columns)

165

In [6]:
#termino en 5 para que no de error en el rango de las columnas
list_df = list()
for i in range( (len(df.columns)) ,8, -1):
    #print("{}-{}".format(i-9,i))
    list_df.append(df.iloc[:,i-9:i])
len(list_df)

157

### Renombre las columnas de cada dataframe

In [7]:
#cambiando el nombre de las fechas a versión generica ( 6 meses train, 3 pred)
cols = ["6m", "5m", "4m", "3m", "2m", "1m", "3p", "2p", "1p"]

for i in range(0, len(list_df)):
    list_df[i].rename(columns=dict(zip(list_df[i].columns[:9], cols)),inplace=True)

/home/seba/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


### Integrando lista de dataframes en un único dataframe

In [8]:
bd = pd.concat(list_df)
bd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7850000 entries, 1690 to 21260067
Data columns (total 9 columns):
6m    float64
5m    float64
4m    float64
3m    float64
2m    float64
1m    float64
3p    float64
2p    float64
1p    float64
dtypes: float64(9)
memory usage: 598.9 MB


### Integrando con dataframe clientes

In [9]:
df_int = pd.merge(bd, df_cli, left_index=True, right_index=True)
df_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7850000 entries, 1690 to 21260067
Data columns (total 21 columns):
6m                 float64
5m                 float64
4m                 float64
3m                 float64
2m                 float64
1m                 float64
3p                 float64
2p                 float64
1p                 float64
DATE_OF_BIRTH      object
GENDER             object
MARITAL_STATUS     object
EDUCATION          object
EDUCATION_LEVEL    object
OCCUPATION         object
CITY               object
LOCALITY           object
FECHA_PROCESO      object
edad               float64
ModoPago           object
edad_r             float64
dtypes: float64(11), object(10)
memory usage: 1.3+ GB


In [10]:
df_int.drop_duplicates(inplace=True)
df_int.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 21 columns):
6m                 457328 non-null float64
5m                 477942 non-null float64
4m                 500223 non-null float64
3m                 522415 non-null float64
2m                 544338 non-null float64
1m                 565878 non-null float64
3p                 586932 non-null float64
2p                 607462 non-null float64
1p                 628205 non-null float64
DATE_OF_BIRTH      867773 non-null object
GENDER             875055 non-null object
MARITAL_STATUS     802517 non-null object
EDUCATION          292549 non-null object
EDUCATION_LEVEL    26972 non-null object
OCCUPATION         544218 non-null object
CITY               682039 non-null object
LOCALITY           682039 non-null object
FECHA_PROCESO      875419 non-null object
edad               867773 non-null float64
ModoPago           875419 non-null object
edad_r             867773 non-null 

In [11]:
df_int.head()

,6m,5m,4m,3m,2m,1m,3p,2p,1p,DATE_OF_BIRTH,...,MARITAL_STATUS,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad,ModoPago,edad_r
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
1690,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
1690,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
1690,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
1690,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0


In [12]:
#variables descartadas
delete_cols = ["DATE_OF_BIRTH", "FECHA_PROCESO", "OCCUPATION", "LOCALITY", "edad"]
df_fin = df_int.drop(delete_cols, axis=1)
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 16 columns):
6m                 457328 non-null float64
5m                 477942 non-null float64
4m                 500223 non-null float64
3m                 522415 non-null float64
2m                 544338 non-null float64
1m                 565878 non-null float64
3p                 586932 non-null float64
2p                 607462 non-null float64
1p                 628205 non-null float64
GENDER             875055 non-null object
MARITAL_STATUS     802517 non-null object
EDUCATION          292549 non-null object
EDUCATION_LEVEL    26972 non-null object
CITY               682039 non-null object
ModoPago           875419 non-null object
edad_r             867773 non-null float64
dtypes: float64(10), object(6)
memory usage: 113.5+ MB


In [13]:
X = pd.get_dummies(df_fin)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 58 columns):
6m                                457328 non-null float64
5m                                477942 non-null float64
4m                                500223 non-null float64
3m                                522415 non-null float64
2m                                544338 non-null float64
1m                                565878 non-null float64
3p                                586932 non-null float64
2p                                607462 non-null float64
1p                                628205 non-null float64
edad_r                            867773 non-null float64
GENDER_F                          875419 non-null uint8
GENDER_M                          875419 non-null uint8
MARITAL_STATUS_Casado             875419 non-null uint8
MARITAL_STATUS_Conviv             875419 non-null uint8
MARITAL_STATUS_Divorciado         875419 non-null uint8
MARITAL_STATUS_No Corres

In [73]:
X.fillna(0, inplace=True) #reemplazo nulos por 0
X = X.astype("uint8") #cambio el tipo de los datos
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 58 columns):
6m                                875419 non-null uint8
5m                                875419 non-null uint8
4m                                875419 non-null uint8
3m                                875419 non-null uint8
2m                                875419 non-null uint8
1m                                875419 non-null uint8
3p                                875419 non-null uint8
2p                                875419 non-null uint8
1p                                875419 non-null uint8
edad_r                            875419 non-null uint8
GENDER_F                          875419 non-null uint8
GENDER_M                          875419 non-null uint8
MARITAL_STATUS_Casado             875419 non-null uint8
MARITAL_STATUS_Conviv             875419 non-null uint8
MARITAL_STATUS_Divorciado         875419 non-null uint8
MARITAL_STATUS_No Corresp         875419 non

In [75]:
X["id_cliente"] = X.index

In [76]:
X.drop_duplicates(inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 59 columns):
6m                                875419 non-null uint8
5m                                875419 non-null uint8
4m                                875419 non-null uint8
3m                                875419 non-null uint8
2m                                875419 non-null uint8
1m                                875419 non-null uint8
3p                                875419 non-null uint8
2p                                875419 non-null uint8
1p                                875419 non-null uint8
edad_r                            875419 non-null uint8
GENDER_F                          875419 non-null uint8
GENDER_M                          875419 non-null uint8
MARITAL_STATUS_Casado             875419 non-null uint8
MARITAL_STATUS_Conviv             875419 non-null uint8
MARITAL_STATUS_Divorciado         875419 non-null uint8
MARITAL_STATUS_No Corresp         875419 non

In [77]:
X.to_csv("../../data/processed/m1.0 - dummy sin churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")

# Training : churn exacto 3 meses

In [14]:
df = pd.read_csv("../../data/processed/m1.0 - dummy sin churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv", index_col="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 59 columns):
6m                                875419 non-null int64
5m                                875419 non-null int64
4m                                875419 non-null int64
3m                                875419 non-null int64
2m                                875419 non-null int64
1m                                875419 non-null int64
3p                                875419 non-null int64
2p                                875419 non-null int64
1p                                875419 non-null int64
edad_r                            875419 non-null int64
GENDER_F                          875419 non-null int64
GENDER_M                          875419 non-null int64
MARITAL_STATUS_Casado             875419 non-null int64
MARITAL_STATUS_Conviv             875419 non-null int64
MARITAL_STATUS_Divorciado         875419 non-null int64
MARITAL_STATUS_No Corresp         875419 non

In [15]:
df["Churn"] = 0
#Regla churn stricta
#df.loc[ (df["3p"] == 0) & (df["2p"] == 0)  &  (df["1p"] == 0) &  (df["1m"] != 0) &  (df.iloc[:,:3].count(axis=1) > 0)  &  (df.iloc[:,1:4].count(axis=1) > 0) &  (df.iloc[:,2:5].count(axis=1) > 0), "Churn"] = 1
#Regla churn no stricta
df.loc[ (df["3p"] == 0) & (df["2p"] == 0)  &  (df["1p"] == 0) &  (df["1m"] != 0) , "Churn"] = 1

In [16]:
df.head()

,6m,5m,4m,3m,2m,1m,3p,2p,1p,edad_r,...,CITY_TUCUMAN,ModoPago_BA,ModoPago_CC,ModoPago_Mix,ModoPago_PP,ModoPago_SJ,ModoPago_TA,ModoPago_TM,id_cliente,Churn
1690,0,0,0,0,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0
1690,1,0,0,0,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0
1690,1,1,0,0,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0
1690,1,1,1,0,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0
1690,1,1,1,1,0,0,0,0,0,5,...,0,0,1,0,0,0,0,0,1690,0


In [18]:
df["Churn"].value_counts() / len(df)

0    0.969396
1    0.030604
Name: Churn, dtype: float64

In [89]:
df.drop("id_cliente", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 59 columns):
6m                                875419 non-null int64
5m                                875419 non-null int64
4m                                875419 non-null int64
3m                                875419 non-null int64
2m                                875419 non-null int64
1m                                875419 non-null int64
3p                                875419 non-null int64
2p                                875419 non-null int64
1p                                875419 non-null int64
edad_r                            875419 non-null int64
GENDER_F                          875419 non-null int64
GENDER_M                          875419 non-null int64
MARITAL_STATUS_Casado             875419 non-null int64
MARITAL_STATUS_Conviv             875419 non-null int64
MARITAL_STATUS_Divorciado         875419 non-null int64
MARITAL_STATUS_No Corresp         875419 non

In [90]:
df.drop(["3p", "2p", "1p"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875419 entries, 1690 to 21260067
Data columns (total 56 columns):
6m                                875419 non-null int64
5m                                875419 non-null int64
4m                                875419 non-null int64
3m                                875419 non-null int64
2m                                875419 non-null int64
1m                                875419 non-null int64
edad_r                            875419 non-null int64
GENDER_F                          875419 non-null int64
GENDER_M                          875419 non-null int64
MARITAL_STATUS_Casado             875419 non-null int64
MARITAL_STATUS_Conviv             875419 non-null int64
MARITAL_STATUS_Divorciado         875419 non-null int64
MARITAL_STATUS_No Corresp         875419 non-null int64
MARITAL_STATUS_Separado           875419 non-null int64
MARITAL_STATUS_Soltero            875419 non-null int64
MARITAL_STATUS_Union Civ          875419 non

In [91]:
df.to_csv("../../data/processed/train/m1.0 - churn estricto - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")